<a href="https://colab.research.google.com/github/tailorfenil/multithreading_and_asyncio/blob/main/Threading_and_asyncIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [330]:
import time
from urllib.parse import urlparse as urlparser
from typing import Set  # Import Set from typing module

In [279]:
# Define test "webs" as graphs
test_graph_1 = {
    "http://site.com/a": ["http://site.com/b", "http://site.com/c","http://site.com/d"],
    "http://site.com/b": ["http://site.com/d"],
    "http://site.com/c": [],
    "http://site.com/d": []
}

test_graph_2 = {
    "http://site.com/a": ["http://site.com/b", "http://othersite.com/e"],
    "http://site.com/b": ["http://site.com/c"],
    "http://site.com/c": [],
    "http://othersite.com/e": ["http://othersite.com/f"],
    "http://othersite.com/f": []
}


In [7]:
class htmlparser:

  def __init__(self,web_crawler_map):
    self.web_crawler_map = web_crawler_map

  def parse(self,url:str) -> list[str]:
    return self.web_crawler_map.get(url,[])

In [377]:

class sequential_crawl:
  def __init__(self,url:str,htmlparser):
    self.url = url
    self.htmlparser = htmlparser
    self.visited = set()
    self.hostname = urlparser(url).hostname

  def crawl(self,url:str):
    if url in self.visited or urlparser(url).hostname != self.hostname:
      return
    self.visited.add(url)
    print(f"Crawling {url}")
    for child in self.htmlparser.parse(url):
      self.crawl(child)

  def run(self,start_url:str) -> Set[str]:
    self.crawl(start_url)
    return self.visited


In [378]:
st_time = time.time()
crawl = sequential_crawl("http://site.com/a",htmlparser(test_graph_1))
visited = crawl.run("http://site.com/a")
end_time = time.time()
print(f"Time taken: {end_time - st_time} seconds")

Crawling http://site.com/a
Crawling http://site.com/b
Crawling http://site.com/d
Crawling http://site.com/c
Time taken: 0.00031638145446777344 seconds


sequential_crawl completes

In [379]:
import threading
class crawl_threading:
  def __init__(self,url:str,htmlparser) -> None:
    self.url=url
    self.visited=set()
    self.htmlparser = htmlparser
    self.hostname = urlparser(url).hostname

  def crawl(self,url:str):
    if url in self.visited or urlparser(url).hostname != self.hostname:
      return
    self.visited.add(url)
    print(f"Crawling {url}")

    threads=[]
    for child in self.htmlparser.parse(url):
      thread = threading.Thread(target=self.crawl,args=(child,))
      thread.start()
      threads.append(thread)

    for thread in threads:
      thread.join()


In [380]:
crawlthreads = crawl_threading("http://site.com/a",htmlparser(test_graph_1))
st_time = time.time()
visite= crawlthreads.crawl("http://site.com/a")
end_time = time.time()
print(f"Time taken: {end_time - st_time} seconds")
print(visited)

Crawling http://site.com/a
Crawling http://site.com/b
Crawling http://site.com/d
Crawling http://site.com/c
Time taken: 0.0030410289764404297 seconds
{'http://site.com/a', 'http://site.com/d', 'http://site.com/b', 'http://site.com/c'}


crawl_threading completes

In [381]:
class crawl_thread_safe:
  def __init__(self,url:str,htmlparser) -> None:
    self.url=url
    self.visited=set()
    self.htmlparser = htmlparser
    self.hostname = urlparser(url).hostname
    self.lock = threading.Lock()

  def crawl(self,url:str):
    with self.lock:
      if url in self.visited or urlparser(url).hostname != self.hostname:
        return
      self.visited.add(url)
      print(f"Crawling {url}")

    threads=[]

    for child in self.htmlparser.parse(url):
      thread = threading.Thread(target=self.crawl,args=(child,))
      thread.start()
      threads.append(thread)

    for thread in threads:
      thread.join()

  def run(self,start_url:str) -> Set[str]:
    self.crawl(start_url)
    return self.visited

In [382]:
st_time = time.time()
crawlthreadsafe = crawl_thread_safe("http://site.com/a",htmlparser(test_graph_1))
visited = crawlthreadsafe.run("http://site.com/a")
end_time = time.time()
print(f"Time taken: {end_time - st_time} seconds")
print(visited)

Crawling http://site.com/a
Crawling http://site.com/b
Crawling http://site.com/d
Crawling http://site.com/c
Time taken: 0.0024394989013671875 seconds
{'http://site.com/a', 'http://site.com/d', 'http://site.com/b', 'http://site.com/c'}


crawl_thread_safe completes

In [383]:
class crawl_thread_safe_concurrrency_limit:
  def __init__(self,url:str,htmlparser,max_threads:int=10) -> None:
    self.url=url
    self.visited=set()
    self.htmlparser = htmlparser
    self.hostname = urlparser(url).hostname
    self.lock= threading.Lock()
    self.semaphore = threading.Semaphore(max_threads)

  def crawl(self,url:str):
    with self.semaphore:
      with self.lock:
        if url in self.visited or urlparser(url).hostname != self.hostname:
          return
        self.visited.add(url)
        print(f"Crawling {url}")

      threads = []
      for child in self.htmlparser.parse(url):
        thread = threading.Thread(target=self.crawl,args=(child,))
        thread.start()
        threads.append(thread)

      for thread in threads:
        thread.join()


In [384]:
crawl_concurrency_limit = crawl_thread_safe_concurrrency_limit("http://site.com/a",htmlparser(test_graph_1))
st_time = time.time()
visited = crawl_concurrency_limit.crawl("http://site.com/a")
end_time = time.time()
print(f"Time taken: {end_time - st_time} seconds")
print(visited)


Crawling http://site.com/a
Crawling http://site.com/b
Crawling http://site.com/d
Crawling http://site.com/c
Time taken: 0.0018458366394042969 seconds
None


crawl_thread_safe_concurrrency_limit completes

In [385]:
from concurrent.futures import ThreadPoolExecutor, wait
class crawlthreadingwithThreadPool:
  def __init__(self,url,htmlparser,max_threads:int=10):
    self.url=url
    self.htmlparser= htmlparser
    self.visited=set()
    self.hostname = urlparser(url).hostname
    self.lock = threading.Lock()
    self.executor = ThreadPoolExecutor(max_workers=max_threads)
    self.futures = []

  def crawl(self,url):
    with self.lock:
      if url in self.visited or urlparser(url).hostname != self.hostname:
        return
      self.visited.add(url)
      print(f"Crawling {url}")

    for child in self.htmlparser.parse(url):
      future = self.executor.submit(self.crawl,child)
      with self.lock:
        self.futures.append(future)


  def run(self,start_url):
    future = self.executor.submit(self.crawl,start_url)
    with self.lock:
      self.futures.append(future)
    wait(self.futures)
    self.executor.shutdown()
    return self.visited

In [400]:
crawlThreadPool= crawlthreadingwithThreadPool("http://site.com/a",htmlparser(test_graph_1))
st_time = time.time()
visited = crawlThreadPool.run("http://site.com/a")
end_time = time.time()
print(f"Time taken: {end_time - st_time} seconds")
print(visited)

'''
But your code does this:

Starts with one future for crawl(a)
That task submits more futures from inside threads
Those new futures are added to self.futures later
But wait(...) is already running and doesn’t know about them
So wait(self.futures) can return before the newly submitted tasks are finished, and the crawler may shut down prematurely.

ou're submitting future tasks asynchronously, and the list of self.futures keeps growing after wait() starts.
But wait(self.futures) only waits on what’s already in that list — it doesn't magically re-evaluate it.

 When You DO Need a Condition
You need a Condition when:

New work is discovered dynamically (like recursive crawling)
You don’t know how many total tasks there will be up front
You want to wait for all dynamic tasks to be “done”, not just submitted
'''

Crawling http://site.com/a
Crawling http://site.com/b
Crawling http://site.com/c
Crawling http://site.com/d
Time taken: 0.0026743412017822266 seconds
{'http://site.com/a', 'http://site.com/d', 'http://site.com/b', 'http://site.com/c'}


"\nBut your code does this:\n\nStarts with one future for crawl(a)\nThat task submits more futures from inside threads\nThose new futures are added to self.futures later\nBut wait(...) is already running and doesn’t know about them\nSo wait(self.futures) can return before the newly submitted tasks are finished, and the crawler may shut down prematurely.\n\nou're submitting future tasks asynchronously, and the list of self.futures keeps growing after wait() starts.\nBut wait(self.futures) only waits on what’s already in that list — it doesn't magically re-evaluate it.\n\n When You DO Need a Condition\nYou need a Condition when:\n\nNew work is discovered dynamically (like recursive crawling)\nYou don’t know how many total tasks there will be up front\nYou want to wait for all dynamic tasks to be “done”, not just submitted\n"

crawlthreadingwithThreadPool completes

In [394]:
class ExecutorBasedCrawler:
    def __init__(self, start_url: str, htmlParser, max_workers: int = 10):
        self.start_url = start_url
        self.htmlParser = htmlParser
        self.hostname = urlparser(start_url).hostname
        self.visited = set()
        self.lock = threading.Lock()
        self.executor = ThreadPoolExecutor(max_workers=max_workers)
        self.active_tasks = 0
        self.condition = threading.Condition()

    def crawl_url(self, url: str):
        with self.lock:
            if url in self.visited or urlparser(url).hostname != self.hostname:
                return
            self.visited.add(url)

        with self.condition:
            self.active_tasks += 1

        print(f"Crawling {url}")
        try:
            children = self.htmlParser.parse(url)
            print(children)
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            with self.condition:
                self.active_tasks -= 1
                self.condition.notify_all()
            return

        for child in children:
            self.executor.submit(self.crawl_url, child)

        with self.condition:
            self.active_tasks -= 1
            self.condition.notify_all()

    def run(self) -> Set[str]:
        self.executor.submit(self.crawl_url, self.start_url)
        with self.condition:
            while self.active_tasks > 0:
                self.condition.wait()
        self.executor.shutdown()
        return self.visited


In [397]:
exec_condition_crawler= ExecutorBasedCrawler("http://site.com/a",htmlparser(test_graph_1))
st_time = time.time()
visited = exec_condition_crawler.run()
end_time = time.time()
print(f"Time taken: {end_time - st_time} seconds")
print(visited)

Crawling http://site.com/a
['http://site.com/b', 'http://site.com/c', 'http://site.com/d']
Crawling http://site.com/b
['http://site.com/d']
Crawling http://site.com/c
[]
Crawling http://site.com/d
[]
Time taken: 0.002387523651123047 seconds
{'http://site.com/a', 'http://site.com/d', 'http://site.com/b', 'http://site.com/c'}


ExecutorBasedCrawler completes

In [408]:
import asyncio
from urllib.parse import urlparse

class HybridAsyncCrawler:
    def __init__(self):
        self.visited = set()
        self.lock = asyncio.Lock()
        self.queue = asyncio.Queue()

    async def downloadUrls(self, url, htmlParser, hostname, loop):
        # Run sync getUrls in a background thread (non-blocking for asyncio)
        next_urls = await loop.run_in_executor(None, htmlParser.parse, url)

        for u in next_urls:
            if urlparse(u).hostname == hostname:
                async with self.lock:
                    if u not in self.visited:
                        self.visited.add(u)
                        await self.queue.put(u)

    async def crawl(self, startUrl: str, htmlParser):
        loop = asyncio.get_event_loop()
        hostname = urlparse(startUrl).hostname
        self.visited = {startUrl}
        await self.queue.put(startUrl)

        tasks = []

        while not self.queue.empty() or tasks:
            while not self.queue.empty():
                url = await self.queue.get()
                task = asyncio.create_task(
                    self.downloadUrls(url, htmlParser, hostname, loop)
                )
                tasks.append(task)

            # Clean up finished tasks
            tasks = [t for t in tasks if not t.done()]
            await asyncio.sleep(0.01)  # Give other coroutines a chance to run

        return list(self.visited)

In [415]:
parser = htmlparser(test_graph_1)
crawler = HybridAsyncCrawler()
st_time = time.time()
# result = asyncio.run(crawler.crawl("http://site.com/a", parser))
result = await crawler.crawl("http://site.com/a", parser)
end_time = time.time()
print(f"Time taken: {end_time - st_time} seconds")
print("Visited URLs:", result)

Time taken: 0.03406953811645508 seconds
Visited URLs: ['http://site.com/a', 'http://site.com/d', 'http://site.com/b', 'http://site.com/c']


HybridAsyncCrawler completes

In [420]:
class AsyncCrawler:
    def __init__(self):
        self.visited = set()
        self.lock = asyncio.Lock()
        self.que = asyncio.Queue()

    async def downloadUrls(self, url, htmlParser, hostname):
        next_urls = htmlParser.parse(url)  # assuming async getUrls ; here we don't have like that , we are not returning async result
        for u in next_urls:
            if urlparse(u).hostname == hostname:
                async with self.lock:
                    if u not in self.visited:
                        self.visited.add(u)
                        await self.que.put(u)

    async def crawl(self, startUrl: str, htmlParser):
        hostname = urlparse(startUrl).hostname
        self.visited = {startUrl}
        await self.que.put(startUrl)

        tasks = []

        while not self.que.empty() or tasks:
            while not self.que.empty():
                url = await self.que.get()
                task = asyncio.create_task(self.downloadUrls(url, htmlParser, hostname))
                tasks.append(task)

            tasks = [t for t in tasks if not t.done()]
            await asyncio.sleep(0.01)  # yield control to event loop

        return list(self.visited)

In [421]:
parser = htmlparser(test_graph_1)
crawler = AsyncCrawler()
st_time = time.time()
# result = asyncio.run(crawler.crawl("http://site.com/a", parser))
result = await crawler.crawl("http://site.com/a", parser)
end_time = time.time()
print(f"Time taken: {end_time - st_time} seconds")
print("Visited URLs:", result)

Time taken: 0.03393411636352539 seconds
Visited URLs: ['http://site.com/a', 'http://site.com/d', 'http://site.com/b', 'http://site.com/c']


AsyncCrawler completes